In [ ]:
!pip install transformers
# !pip install nlpaug
!git clone https://github.com/joseph1723/CS376_Final_Project.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 30.8 MB/s 
     |████████████████████████████████| 6.6 MB 51.7 MB/s 
     |████████████████████████████████| 596 kB 51.6 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'CS376_Final_Project'...
remote: Enumerating objects: 298, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 298 (delta 13), reused 33 (delta 10), pack-reused 254
Receiving objects: 100% (298/298), 72.00 MiB | 17.52 MiB/s, done.
Resolving deltas: 100% (135/135), done.


In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertTokenizer, BertForSequenceClassification, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

if torch.cuda.is_available() :
  print("CUDA")
  device = torch.device("cuda")
  model.to(device)
else :
  print("CPU")
  device = torch.device("cpu")
  model.to(device)

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

CUDA


In [ ]:
class TestDataset(Dataset) :
  #Dataset - English/typo-added/labeled
  def __init__(self, df) :
    self.df = df
  
  def __len__(self) :
    return len(self.df)
  
  def __getitem__(self, idx):
    text = self.df.iloc[idx, 0]
    item = (self.df.iloc[idx, 1], self.df.iloc[idx, 2])
    return text, item

Hyperparameters

In [ ]:
train_rate, test_rate = 0.9, 0.09
itr = 1
p_itr = 100
epochs = 5
batch = 10

In [ ]:
total_df = pd.read_csv('/content/CS376_Final_Project/augmented_data/Dataset_aug_complex_10424_spellcheck_rnn.csv', sep=',')
#total_df = pd.read_csv('augmented_data/Dataset_aug_complex_10424_original.csv', sep=',')

total_df.dropna(inplace=True)
total_df = total_df[["text", "label", "text_corrected"]]
#total_df["label"] = [1 if i == "nothate" else 0 for i in total_df["label"]]
print(total_df)
total_dataset = TestDataset(total_df)
total_loader = DataLoader(total_dataset, batch_size=batch, shuffle=True)

                                                    text  label  \
0      just called this hotch a cum dumpster on the t...      0   
1      This dress looks fudk7nf bad on me and I look ...      1   
2      If they people don't react well, to you coming...      1   
3      "There is no correlation between black populat...      0   
4                F6sk yeah, I would kill for some fries.      1   
...                                                  ...    ...   
10419  would you like to be called brown? Then stop u...      1   
10420  i hate being a lesbn#1 living in this country,...      1   
10421  The curfew you wanted but now you don't, make ...      1   
10422  I love how HE has to come on to this forum, ch...      0   
10423  would you like to be called a q7eeT? Then stop...      1   

                                          text_corrected  
0      just called this hotch a cum dumpster on the t...  
1      This daessyc looks fudk7nf bad on me and I loo...  
2      If they peo

In [ ]:
#Train Set
test_df, train_df, _ = np.split(total_df, [int(test_rate*len(total_df)), int(test_rate*len(total_df) + train_rate*len(total_df))])
print(len(test_df), len(train_df))
train_dataset = TestDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
test_dataset = TestDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=batch, shuffle=True)

938 9381


In [ ]:
optimizer = Adam(model.parameters(), lr=1e-6)

total_loss = 0
total_len = 0
total_correct = 0
total_preds = []
X=1

model.train()
for epoch in range(epochs):
    
    for text, item in train_loader:
        label, text_corrected = item

        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text_corrected]
        MAX_LEN = max(len(e) for e in encoded_list)
        padded_list =  [e + [0] * (MAX_LEN-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels, return_dict=False)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=X)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()

        total_preds.append(pred)
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0
        itr+=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch 1/5] Iteration 100 -> Train Loss: 0.6940, Accuracy: 0.508
[Epoch 1/5] Iteration 200 -> Train Loss: 0.6868, Accuracy: 0.548
[Epoch 1/5] Iteration 300 -> Train Loss: 0.6724, Accuracy: 0.585
[Epoch 1/5] Iteration 400 -> Train Loss: 0.6730, Accuracy: 0.589
[Epoch 1/5] Iteration 500 -> Train Loss: 0.6675, Accuracy: 0.583
[Epoch 1/5] Iteration 600 -> Train Loss: 0.6698, Accuracy: 0.584
[Epoch 1/5] Iteration 700 -> Train Loss: 0.6703, Accuracy: 0.603
[Epoch 1/5] Iteration 800 -> Train Loss: 0.6623, Accuracy: 0.598
[Epoch 1/5] Iteration 900 -> Train Loss: 0.6477, Accuracy: 0.618
[Epoch 2/5] Iteration 1000 -> Train Loss: 0.6398, Accuracy: 0.626
[Epoch 2/5] Iteration 1100 -> Train Loss: 0.6141, Accuracy: 0.635
[Epoch 2/5] Iteration 1200 -> Train Loss: 0.6497, Accuracy: 0.614
[Epoch 2/5] Iteration 1300 -> Train Loss: 0.6206, Accuracy: 0.646
[Epoch 2/5] Iteration 1400 -> Train Loss: 0.6118, Accuracy: 0.632
[Epoch 2/5] Iteration 1500 -> Train Loss: 0.6073, Accuracy: 0.670
[Epoch 2/5] Iterati

In [ ]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 33.3 MB/s 


In [ ]:
from torchmetrics import F1Score

model.eval()

total_loss = 0
total_len = 0
total_correct = 0
total_preds = []

for text, item in test_loader:
    label, text_corrected = item

    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text_corrected]
    MAX_LEN = max(len(e) for e in encoded_list)
    padded_list =  [e + [0] * (MAX_LEN-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels, return_dict=False)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    total_preds.append(pred)

    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

labels = torch.tensor(test_df['label']).to(device)
total_preds = torch.cat(total_preds).to(device)
f1 = F1Score(num_classes=2).to(device)
F1_Score = f1(total_preds, labels)
print('F1 Score : ', F1_Score)

print('Test accuracy: ', total_correct / total_len)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


F1 Score :  tensor(0.4947, device='cuda:0')
Test accuracy:  0.7356076759061834


# Accuracy (epoch=5) : 0.735
# F1-Score : 0.4947